# Examples of Binomial Coefficients

# Showing also how to use the Bokeh package to plot multiple distributions in one plot.



This is a solution to the homework problem in "Probability HW1" for computing the $\binom{n}{k}\  \text{for} \ n=0\ldots10, 0 \le k \le n$ to  create “Pascal’s Triangle.” and plotting the values of $\binom{n}{k}$ for each value of n.

JMA Feb 2024

In [86]:
# An example of the typical imports for data science notebook
# not all will be used here

# Import python standard library
import re, os, sys, time
import math
from pathlib import Path

# Import array and dataframe packages
import numpy as np
from numpy.random import default_rng
import pandas as pd

# Import ML & stats packages
import sklearn as sk
import scipy as sp     # for the binom(n,k) function

# Import the bokeh python wrappers for javascript plots
#  - a preferred visualization tool
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, VBar
from bokeh.io import output_notebook
output_notebook()

# Run the iterations up to n = 10
sample_max = 10

In [87]:
# We'll need this when we center the plots of the binomial coefficients around
# their maximums.

def is_even(n):
  'test an integer'
  return (float(n // 2) == n/2)  # test for equality requires both to be the same type

In [88]:
# "Normalize_and_center" rescales the coefficients to sum to 1, and centering
# makes them symmetric around k.
def binomial_coef_distribution(sample_size = 30, normalize_and_center=False):
  'Generate a vector of one row in Pascals Triangle'
  all_combinations = np.array(list(range(sample_size+1)))
  coefs = [sp.special.comb(sample_size, c) for c in all_combinations]
  if normalize_and_center:
    max_coef = max(coefs)
    max_coef_index = coefs.index(max_coef)
    if is_even(len(all_combinations)):
      all_combinations = all_combinations - (all_combinations[max_coef_index] + all_combinations[max_coef_index+1])/2
    else:
      all_combinations = all_combinations - (all_combinations[max_coef_index])
    coefs = np.array(coefs) / sum(coefs)
  return all_combinations, coefs

binomial_coef_distribution(9, True)


(array([-4.5, -3.5, -2.5, -1.5, -0.5,  0.5,  1.5,  2.5,  3.5,  4.5]),
 array([0.00195312, 0.01757812, 0.0703125 , 0.1640625 , 0.24609375,
        0.24609375, 0.1640625 , 0.0703125 , 0.01757812, 0.00195312]))

In [89]:
import colorcet as cc

from bokeh.models import ColumnDataSource, FixedTicker, PrintfTickFormatter
from bokeh.plotting import figure, show


# Pick from the color palette of 256 a list of sample size
palette = [cc.rainbow[i * math.floor(256/sample_max)] for i in range(sample_max)]

# y range is an enumeration of column names
p = figure(y_range=(0,max(binomial_coef_distribution(sample_max)[1])),
           width=1000,
           x_range=(0, sample_max),
           title = "Binomial Coefficients for varying values of n")

for n in range(1, sample_max+1):
    # Generate the binom coefficients for this k
    x, binom_coefs = binomial_coef_distribution(n)
    bc_source = ColumnDataSource(data = {'x': x, 'y': binom_coefs})
    p.line(x='x', y='y', source=bc_source, color=palette[n-1])
    p.square(x='x', y='y', source=bc_source, color=palette[n-1], size=3)
p.background_fill_color = "#efefef"

show(p)

### What shape do binomial coefficients take as n gets large?

As the size of the sample increases, the sequence of binomial coefficients for some _n_ approaches a continuous distribution.  We show this by normalizing the coefficients and plotting them together.

In [90]:
# y range is an enumeration of column names
p = figure(y_range=(-sample_max * 0.02,0.55), width=1000, x_range=(-sample_max//2, sample_max//2))

for n in range(1, sample_max+1):
    # Generate the binom coefficients for this k
    x, binom_coefs = binomial_coef_distribution(n, normalize_and_center=True)
    # source.add(y, cat)
    # p.patch('x', cat, color=palette[i], alpha=0.6, line_color="black", source=source)
    bc_source = ColumnDataSource(data = {'x': x, 'y': binom_coefs - n * 0.02})
    # p.cross(x='x', y='y', size = 10, source=bc_source)
    p.line(x='x', y='y', source=bc_source, color=palette[n-1])
    p.square(x='x', y='y', source=bc_source, color=palette[n-1], size=3)

# Remove y axis - this is cumbersome
p.yaxis.minor_tick_line_color = None
p.yaxis.major_tick_line_color = None
p.yaxis.axis_line_color = None
p.yaxis.major_label_text_font_size = '0pt'

p.ygrid.grid_line_color = None
p.background_fill_color = "#efefef"
show(p)

### Fitting the normalized binomial coefficients to a Gaussian distribution.

Let's test if if the approximation looks good.  
The function gaussian_kde approximates the points from the binomial coefficients with a Gaussian distribution function.  One could increase n and see how the fit progresses.

In [91]:

from scipy.stats import gaussian_kde

p = figure(width = 1000, height = 300)
p.square(x=x, y=binom_coefs/sum(binom_coefs), line_color='navy')
# In normal usage gaussiann kde takes a sample.  To simulate that,
# we use the weights argument to scale each sample point by it's equivalent count

p.line(x=x, y=gaussian_kde(x, weights=binom_coefs/ sum(binom_coefs)).pdf(x), line_color='red')

show(p)

## Plot a pmf - for the roll of a pair of dice

This is a demonstration of how to plot a pmf, and shows a typical distribution

In [92]:
# The role of one die, extended to the dimensions of the two r.v.s
distribution = np.array(range(1,7)).reshape((1,6)) * np.ones((6,6)) # @ np.array(range(1,7)).reshape((1,6))
distribution

array([[1., 2., 3., 4., 5., 6.],
       [1., 2., 3., 4., 5., 6.],
       [1., 2., 3., 4., 5., 6.],
       [1., 2., 3., 4., 5., 6.],
       [1., 2., 3., 4., 5., 6.],
       [1., 2., 3., 4., 5., 6.]])

In [93]:
# The r.vs for the events for the role of two dice
dice_rv = distribution.T + distribution
dice_rv

array([[ 2.,  3.,  4.,  5.,  6.,  7.],
       [ 3.,  4.,  5.,  6.,  7.,  8.],
       [ 4.,  5.,  6.,  7.,  8.,  9.],
       [ 5.,  6.,  7.,  8.,  9., 10.],
       [ 6.,  7.,  8.,  9., 10., 11.],
       [ 7.,  8.,  9., 10., 11., 12.]])

In [94]:
# count the events. This is a way to build a histogram of individual rv values.
dice_histogram = pd.DataFrame(dice_rv.ravel(), columns=['rv_s']).value_counts().sort_index().reset_index()
dice_histogram.columns = ['rv_s', 'counts']    # Is there an easier way to add a column name for value_counts ?
dice_histogram = dice_histogram.assign(distribution = dice_histogram['counts'].values/sum(dice_histogram['counts'].values))
dice_histogram

,rv_s,counts,distribution
0,2.0,1,0.027778
1,3.0,2,0.055556
2,4.0,3,0.083333
3,5.0,4,0.111111
4,6.0,5,0.138889
5,7.0,6,0.166667
6,8.0,5,0.138889
7,9.0,4,0.111111
8,10.0,3,0.083333
9,11.0,2,0.055556


In [95]:
hist = ColumnDataSource(dice_histogram)
p = figure(width = 1000, height = 300)
p.circle(x='rv_s', y='distribution', line_color='navy', source=hist , size= 10)
glyph = VBar(x='rv_s', top='distribution', bottom=0, width=0.05, fill_color='blue')
p.add_glyph(hist, glyph)
show(p)